In [1]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [2]:
stack = setup_rds_stack(log_level="INFO")

2025-03-06 14:22:55.738 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-06 14:22:55.744 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_f3upeb9k...


In [3]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [4]:
private_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "private"
mock_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "mock"

private_path.mkdir(parents=True, exist_ok=True)
mock_path.mkdir(parents=True, exist_ok=True)

with open(RDS_NOTEBOOKS_PATH /"data/dataset-1/private/data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(RDS_NOTEBOOKS_PATH / "data/dataset-1/mock/data.csv", "w") as f:
    f.write("1,2,3")


In [5]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_path,  # MUST EXIST
    mock_path=mock_path,
)

In [6]:
do_client.datasets

[Dataset
   uid: 79e55aa9-3a3e-4229-b85a-a603aef824a2
   created_at: 2025-03-06T13:22:55.839706Z
   updated_at: 2025-03-06T13:22:55.839711Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

# Use dataset as DS

In [7]:
ds_client.datasets

[Dataset
   uid: 79e55aa9-3a3e-4229-b85a-a603aef824a2
   created_at: 2025-03-06T13:22:55.839706Z
   updated_at: 2025-03-06T13:22:55.839711Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

In [8]:
ds_client.dataset.get("dataset-1").get_mock_path()


PosixPath('/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_f3upeb9k/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [9]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_f3upeb9k/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [10]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

└── data.csv


True

In [11]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template

In [12]:
my_file = RDS_NOTEBOOKS_PATH / "data" / "code1.py"

In [13]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Overwriting /Users/azizwork/Workspace/rds/notebooks/data/code1.py


In [14]:
dataset.name

'dataset-1'

In [15]:
job = ds_client.jobs.submit(
    user_code_path=my_file,
    dataset_name = dataset.name
)
job


Job
  uid: a6877df0-8115-443e-b2ae-ebcef2c10c1c
  created_at: 2025-03-06T13:22:56.558320Z
  updated_at: 2025-03-06T13:22:56.558321Z
  name: agile_graph_bujb
  description: None
  user_code_id: a21f7a42-99b3-4e3d-abf4-aee01f633aad
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c
  dataset_name: dataset-1

In [16]:
ds_client.run_private(job)

2025-03-06 14:22:56.770 | WARNING  | syft_rds.client.rds_client:run_private:117 - Running job without docker is not secure


╭───────────────────────────── Job Configuration ──────────────────────────────╮
│ Starting job                                                                 │
│ Execution: python code1.py                                                   │
│ Dataset Dir.:  .../data_owner@test.openmined.org/private/datasets/dataset-1  │
│ Output Dir.:   .../tmp/syft-rds-jobs/a6877df08115443eb2aeebcef2c10c1c/output │
│ Timeout:  60s                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

Job
  uid: a6877df0-8115-443e-b2ae-ebcef2c10c1c
  created_at: 2025-03-06T13:22:56.558320Z
  updated_at: 2025-03-06T13:22:56.811217Z
  name: agile_graph_bujb
  description: None
  user_code_id: a21f7a42-99b3-4e3d-abf4-aee01f633aad
  tags: []
  user_metadata: {}
  status: job_run_finished
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c
  dataset_name: dataset-1

In [17]:
do_job = do_client.jobs.get_all()[-1]

In [18]:
from syft_rds.client.utils import PathLike
from syft_rds.models.models import Job, JobStatus, JobUpdate


In [19]:
from syft_rds.models.models import JobErrorKind


do_client.rpc.jobs.update(
        JobUpdate(
            uid=do_job.uid,
            status=JobStatus.queued,
            error=JobErrorKind.no_error
        ))

Job
  uid: a6877df0-8115-443e-b2ae-ebcef2c10c1c
  created_at: 2025-03-06T13:22:56.558320Z
  updated_at: 2025-03-06T13:22:57.068734Z
  name: agile_graph_bujb
  description: None
  user_code_id: a21f7a42-99b3-4e3d-abf4-aee01f633aad
  tags: []
  user_metadata: {}
  status: queued
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c
  dataset_name: dataset-1

In [20]:
do_client.jobs.get_all()

[Job
   uid: a6877df0-8115-443e-b2ae-ebcef2c10c1c
   created_at: 2025-03-06T13:22:56.558320Z
   updated_at: 2025-03-06T13:22:57.068734Z
   name: agile_graph_bujb
   description: None
   user_code_id: a21f7a42-99b3-4e3d-abf4-aee01f633aad
   tags: []
   user_metadata: {}
   status: queued
   error: no_error
   output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c
   dataset_name: dataset-1]

In [21]:
def share_results(self, job: Job, job_output_path: PathLike) -> Job:
    self.local_store.jobs.share_result_artifacts(job, job_output_path)

    job = self.rpc.jobs.update(
        JobUpdate(
            uid=job.uid,
            status=JobStatus.shared,
        )
    )
    return job

In [22]:
private_path = do_client.dataset.get(do_job.dataset_name).get_private_path()

In [23]:
do_job = do_client.run_private(do_job)

2025-03-06 14:22:57.601 | WARNING  | syft_rds.client.rds_client:run_private:117 - Running job without docker is not secure


╭───────────────────────────── Job Configuration ──────────────────────────────╮
│ Starting job                                                                 │
│ Execution: python code1.py                                                   │
│ Dataset Dir.:  .../data_owner@test.openmined.org/private/datasets/dataset-1  │
│ Output Dir.:   .../tmp/syft-rds-jobs/a6877df08115443eb2aeebcef2c10c1c/output │
│ Timeout:  60s                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

In [24]:
do_job

Job
  uid: a6877df0-8115-443e-b2ae-ebcef2c10c1c
  created_at: 2025-03-06T13:22:56.558320Z
  updated_at: 2025-03-06T13:22:57.694449Z
  name: agile_graph_bujb
  description: None
  user_code_id: a21f7a42-99b3-4e3d-abf4-aee01f633aad
  tags: []
  user_metadata: {}
  status: job_run_finished
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c
  dataset_name: dataset-1

In [25]:
do_client.share_results(do_job)

file:////private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_f3upeb9k/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/a6877df0-8115-443e-b2ae-ebcef2c10c1c


/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_nn_k_590/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/8de5e43a-94ef-45f8-837c-e114c06a7d2c